# Week 14-  Class 12
This notebook have the most important exercises for theoretical class, which covers chapter 11 from the book. The labs from https://github.com/fchollet/deep-learning-with-python-notebooks are a guide for this notebook, mainly the labs from chapter 11 and 12

####1. Downloading the data

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

####2. Creating the dataset

In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras

batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    #os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    #20% for validation
    num_val_samples = int(0.2 * len(files))
    #select last num_val_samples (files) from array
    val_files = files[-num_val_samples:]
    #move last 20% of training files to validation
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

In [ ]:
for x, y in train_ds:
  print(x)
  print ('\n')
  print(y)
  break

**Dummy Vectorization example**

Understand The TextVectorization layer in keras

In [ ]:
import string
# The TextVectorization layer is just like this class example created here

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)



Exercise

In [ ]:

# initialize the class
<your-answer>


dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]

#create the vocabulary from the dataset
<your-answer>


Exercise

In [ ]:
test_sentence = "I write, rewrite, and still rewrite again"
#encode the test_sentence
<your-answer>


Exercise

In [ ]:
#decode the test_sentence
<your-answer>


Optional example: if you need to pre process your text data in a specific way that the TextVectorizarion layer doesn't do in keras, you can define your own. 

In [ ]:
#this will be skipped in lab class
from tensorflow.keras.layers import TextVectorization
import re
import string
import tensorflow as tf

#initialize
text_vectorization = TextVectorization(
    output_mode="int",
)

#your custom functions

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)

#add the functions in the layer parameters
text_vectorization = TextVectorization(
    output_mode="int",
    #added here
    standardize=custom_standardization_fn,
    #added here
    split=custom_split_fn,
)

#check results with example
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)


####3. Vectorizing

In [ ]:
#Data Transformation

from tensorflow.keras import layers
#resulting in a tensor of shape [batch_size, output_sequence_length], can be padded or truncated
max_length = 600
#The maximum size of the vocabulary for this layer.
max_tokens = 20000
#if output mode is 'tf-idf'
    #It weights a given term by taking “term frequency,” 
    #how many times the term appears in the current document, 
    #and dividing it by a measure of “document frequency,” 
    #which estimates how often the term comes up across the dataset
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

#just like fit
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

####Create Baseline model

Exercise

In [ ]:
max_tokens=20000
hidden_dim=16

inputs = <your-answer> 
x = <your-answer> #dense layer with relu
x = <your-answer> #dropout 0.5
outputs = <your-answer> #activation function is sigmoid
model = keras.Model(inputs, outputs)
<your-answer> #the optimizer is rmsprop

<your-answer> #print the summary of the model

<your-answer> #create callbacks array saving the best model

<your-answer> #train the model

<your-answer> #load the best model

<your-answer> #evaluate and print the metric

####Main model (Embedding layer)

In [ ]:
#this layer takes word index and transform that into a word vector of [1,output_dim] size 
#input=[batch_size, sequence_length]
#output=[batch_size,sequence_length, embedding_ dimensionality]
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
#Bidirectional: two RNN layers running in parallel, 
#with one processing the tokens in their natural order, 
#and the other processing the same tokens in reverse
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_emb = keras.Model(inputs, outputs)
model_emb.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model_emb.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]
model_emb.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callbacks)
model_emb = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model_emb.evaluate(int_test_ds)[1]:.3f}")

####Export and predict model

Exercise

In [ ]:
#Based on the doc: https://keras.io/examples/nlp/pretrained_word_embeddings/
# predict the sentence: 'This movie is complex and confusing, i didn't like it'

<your-answer>

In [ ]:
#Clustering the embedding space
#https://stackoverflow.com/questions/51235118/how-to-get-word-vectors-from-keras-embedding-layer



#### Improvement step: Using pretrained word embeddings (optional)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

####Main model 2: Transformer for text classification

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [ ]:


vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
#order
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
#context
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

In [ ]:
#https://arxiv.org/abs/1907.11692
#https://github.com/keras-team/keras-nlp/blob/v0.5.1/keras_nlp/models/roberta/roberta_backbone.py#L36